<h1>Undersampling in Markov chains</h1>

The purpose of this notebook is to demonstrate how non-converter journeys in a typical multi-touch attribution model can be undersampled to speed up Markov chain training.

## 1. Read-in sample data base <a name="se:data"></a>

A sample data set was downloaded from **Kaggle** ([here](https://www.kaggle.com/hughhuyton/multitouch-attribution-modelling#Conclusions)) which is then sorted by cookie ID and dates in ascending order. The sample data set consists of `240107` total journeys of which `17639` joruneys end in conversion while the other `222468` journeys do not convert. This data set can represent a real-world scenario in which the underlying customer journeys data is unbalanced with a non-conversion majority class. 

In [12]:
import copy
import pandas as pd
import csv
import random
import numpy as np
import os
import inspect

In [20]:
#read Kaggle data
curr_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(curr_dir)
with open(os.path.join(parent_dir, '../Data/attribution_data_kaggle.csv'), 
          newline = '', 
          encoding = "utf-8-sig") as csvfile:
    sample_data = list(csv.reader(csvfile))
pd.DataFrame(sample_data, columns = ['cookie', 'time', 'conversion', 'channel', 'Touch']).head(10)

,cookie,time,conversion,channel,Touch
0,cookie,time,conversion,channel,Touch
1,00000FkCnDfDDf0iC97iC703B,2018-07-03T13:02:11Z,0,Instagram,1
2,00000FkCnDfDDf0iC97iC703B,2018-07-17T19:15:07Z,0,Online Display,2
3,00000FkCnDfDDf0iC97iC703B,2018-07-24T15:51:46Z,0,Online Display,3
4,00000FkCnDfDDf0iC97iC703B,2018-07-29T07:44:51Z,0,Online Display,4
5,0000nACkD9nFkBBDECD3ki00E,2018-07-03T09:44:57Z,0,Paid Search,1
6,0000nACkD9nFkBBDECD3ki00E,2018-07-03T23:36:49Z,0,Paid Search,2
7,0000nACkD9nFkBBDECD3ki00E,2018-07-10T22:24:41Z,0,Paid Search,3
8,0000nACkD9nFkBBDECD3ki00E,2018-07-10T22:24:44Z,0,Paid Search,4
9,0000nACkD9nFkBBDECD3ki00E,2018-07-11T21:21:46Z,0,Paid Search,5


The above data set is not in the desired format. To format the data where each cookie ID corresponds to a certain conversion path, data is aggregated according to the following. Also note that, for later uses, two separate path lists are created for journeys that end in conversion/loss.

In [24]:
#format fata into journey format
path = []
path_no_conv = []
path_conv = []
temp = []
for i in range(1, len(sample_data) - 1):
    if sample_data[i][0] ==  sample_data[i + 1][0]:
        temp.append(sample_data[i][3])
    else:
        temp.append(sample_data[i][3])
        if sample_data[i][2] =='1':
            temp.append('Conv')
            path_conv.append(temp)
        else:
            temp.append('NoConv')
            path_no_conv.append(temp)
        path.append(temp)
        temp = []

In [35]:
#summarize data
print(len(path), '\t total journeys')
print(len(path_conv), '\t conv')
print(len(path_no_conv), '\t no conv')

240107 	 total journeys
17639 	 conv
222468 	 no conv


**Peak at the path data**

In [31]:
print(*path[:5], sep='\n')

['Instagram', 'Online Display', 'Online Display', 'Online Display', 'NoConv']
['Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'NoConv']
['Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'NoConv']
['Instagram', 'NoConv']
['Paid Search', 'NoConv']


**Peak at the paths that lead to conversion only**

In [33]:
print(*path_conv[:5], sep='\n')

['Paid Search', 'Conv']
['Facebook', 'Conv']
['Facebook', 'Conv']
['Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Instagram', 'Online Video', 'Online Video', 'Online Video', 'Online Video', 'Conv']
['Facebook', 'Facebook', 'Conv']


**Peak at the paths that do not convert**

In [34]:
print(*path_no_conv[:5], sep='\n')

['Instagram', 'Online Display', 'Online Display', 'Online Display', 'NoConv']
['Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'NoConv']
['Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'Paid Search', 'NoConv']
['Instagram', 'NoConv']
['Paid Search', 'NoConv']


## 2. Train the Markov chain model <a name="se:markov"></a>

The `transition_matrix` function given a list of paths and states (which here represent channels) can calculate the Markov's frequency and transition probability matrix. This is done by enumerating the transitions between any combination of two states. To calculate transition probabilities, each cell of the frequency matrix (calculated previously) is divided by the sum of its row frequencies. 

In [36]:
#function create markow chain frequencies and probabilities given the journeys
def transition_matrix(transitions, states):
    n = len(states)
    state_dict = {}
    for i in range(n):
        state_dict[states[i]] = i 
    
    #define a matrix for state --> state transitions
    M = [[0] * n for _ in range(n)]
    
    #enumerate frequencies
    for x in range(len(transitions)):
        for (i, j) in zip(transitions[x], transitions[x][1:]):
            M[state_dict[i]][state_dict[j]] += 1
    
    #keep the frequency matrix as a copy 
    N = copy.deepcopy(M)
    
    #convert to probabilities
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    
    return M, N

Define the states in all transitions and evaluate the frequency and transition probability matrix

In [37]:
#Markov chain on path data
states = ['Instagram', 'Online Display', 'Paid Search', 'Facebook', 'Online Video', 'Conv', 'NoConv']
prob_mat, freq_mat = transition_matrix(path, states)

In [38]:
print('Frequency Matrix')
pd.DataFrame(freq_mat, columns = states, index = states).round(2)

Frequency Matrix


,Instagram,Online Display,Paid Search,Facebook,Online Video,Conv,NoConv
Instagram,12001,1405,2508,28014,1673,2244,27356
Online Display,1497,24798,5496,3481,1169,2139,32473
Paid Search,3431,6281,58453,7883,3746,4547,67098
Facebook,28012,3198,5977,65576,3864,5301,63813
Online Video,1642,1121,2796,3939,68668,3408,31728
Conv,0,0,0,0,0,0,0
NoConv,0,0,0,0,0,0,0


In [39]:
print('Transition Matrix')
pd.DataFrame(prob_mat, columns = states, index = states).round(2)

Transition Matrix


,Instagram,Online Display,Paid Search,Facebook,Online Video,Conv,NoConv
Instagram,0.16,0.02,0.03,0.37,0.02,0.03,0.36
Online Display,0.02,0.35,0.08,0.05,0.02,0.03,0.46
Paid Search,0.02,0.04,0.39,0.05,0.02,0.03,0.44
Facebook,0.16,0.02,0.03,0.37,0.02,0.03,0.36
Online Video,0.01,0.01,0.02,0.03,0.61,0.03,0.28
Conv,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NoConv,0.00,0.00,0.00,0.00,0.00,0.00,0.00


<span style="color:red">Note that a true transition matrix for a Markov chain with absorbing state should have transition probabilities of one for those states</span>, however, these can be set after the calculation above

## 3. Undersampling non-converters <a name="se:undersampling"></a>

The goal is to undersample paths that do not result in conversion but ensure that the Markov chain trained on the undersampled data will still represent the true transition probabilities, thus attributions. To that end, data is partitioned to conversion and non-conversion journeys which are stored in `path_conv` and `path_no_conv` lists, respectively.Then, the `path_no_conv` list is undersampled and frequency matrices are caclulated for both data sets. 

In [40]:
#train Markov chain on a subset of data consisting only of paths that lead to conversion
prob_mat_conv, freq_mat_conv = transition_matrix(path_conv, states)

print('Conversion Frequency Matrix')
pd.DataFrame(freq_mat_conv, columns = states, index = states).round(2)

Conversion Only Frequency Matrix


,Instagram,Online Display,Paid Search,Facebook,Online Video,Conv,NoConv
Instagram,1877,157,276,4354,249,2244,0
Online Display,151,2115,523,322,130,2139,0
Paid Search,298,536,4804,751,406,4547,0
Facebook,4277,303,658,10342,583,5301,0
Online Video,225,109,324,518,12519,3408,0
Conv,0,0,0,0,0,0,0
NoConv,0,0,0,0,0,0,0


In [42]:
#train Markov chain on an undersampled subset of data consisting only of paths that do not lead to conversion
undersample_ratio = 0.1
path_no_conv_sample = random.sample(path_no_conv, int(len(a_no_conv) * undersample_ratio))

prob_mat_no_conv, freq_mat_no_conv = transition_matrix(path_no_conv_sample, states)
print('Non-Conversion Frequency Matrix')
pd.DataFrame(freq_mat_no_conv, columns = states, index = states).round(2)

Non-Conversion Only Frequency Matrix


,Instagram,Online Display,Paid Search,Facebook,Online Video,Conv,NoConv
Instagram,1011,126,212,2317,147,0,2655
Online Display,147,2344,529,296,109,0,3291
Paid Search,322,576,5210,755,323,0,6710
Facebook,2293,270,560,5400,328,0,6369
Online Video,140,97,223,359,5414,0,3221
Conv,0,0,0,0,0,0,0
NoConv,0,0,0,0,0,0,0


## 4. Adjusting the frequencies <a name="se:adjustment"></a>

After evaluating the frequency matrix for an undersampled non-conversion data, to estimate the true transition matrix, the frequencies must be adjusted back to represent true class proportions. Therefore, the non-conversion frequency matrix is multiplied by the reverse undersampling ratio. Aggregating the adjusted non-conversion frequency matrix and the conversion frequency matrix should be able to estimate the true frequency matrix.

In [45]:
#adjusting the non-conversion frequency matrix
#aggregatomg conversion and adjusted non-conversion frequency matrix
freq_mat_adjusted = np.add(np.array(freq_mat_no_conv) * (1 / undersample_ratio), freq_mat_conv).tolist()

In [46]:
#use adjusted frequencies reevaluate transition probabilities
#Note that this markow chain is created with all conversion data and adjusted undersampled no-conversion data. 
prob_mat_adjusted=copy.deepcopy(freq_mat_adjusted)

for row in prob_mat_adjusted:
    s = sum(row)
    if s > 0:
        row[:] = [f/s for f in row]

print('Adjusted Undersampled Transition Probabilities')
pd.DataFrame(prob_mat_adjusted, columns = states, index = states).round(2)

Adjusted Undersampled Transition Probabilities


,Instagram,Online Display,Paid Search,Facebook,Online Video,Conv,NoConv
Instagram,0.16,0.02,0.03,0.37,0.02,0.03,0.36
Online Display,0.02,0.35,0.08,0.05,0.02,0.03,0.45
Paid Search,0.02,0.04,0.38,0.06,0.02,0.03,0.45
Facebook,0.16,0.02,0.04,0.37,0.02,0.03,0.37
Online Video,0.01,0.01,0.02,0.04,0.60,0.03,0.29
Conv,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NoConv,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## 5. Comparison with original Markov model <a name="comparison"></a>
To compare the results, the difference between the original transition probabilities matrix, and the adjusted undersampled transition probability matrix is calculated.

In [48]:
origin_prob_df = pd.DataFrame(prob_mat, columns = states, index = states).round(2) 
adjusted_undersampled_prob_df = pd.DataFrame(prob_mat_adjusted, columns = states, index = states).round(2)

origin_prob_df - adjusted_undersampled_prob_df

,Instagram,Online Display,Paid Search,Facebook,Online Video,Conv,NoConv
Instagram,0.0,0.0,0.00,0.00,0.00,0.0,0.00
Online Display,0.0,0.0,0.00,0.00,0.00,0.0,0.01
Paid Search,0.0,0.0,0.01,-0.01,0.00,0.0,-0.01
Facebook,0.0,0.0,-0.01,0.00,0.00,0.0,-0.01
Online Video,0.0,0.0,0.00,-0.01,0.01,0.0,-0.01
Conv,0.0,0.0,0.00,0.00,0.00,0.0,0.00
NoConv,0.0,0.0,0.00,0.00,0.00,0.0,0.00


This notebook demonstrates a framework for undersampling when training a Markov chain over a **tall** data set is required. To remove any bias from the adjustment estimation, one can repeat the process for different sequence of random numbers and take a sample average of the results. 